<h1><font style="background: linear-gradient(to right,gold, red,pink, green);-webkit-background-clip: text;-webkit-text-fill-color: transparent;">SDXL模型的使用</font></h1>

**使用类来避免重复加载模型**


需要`32G` gpu环境。
![](https://ai-studio-static-online.cdn.bcebos.com/18040721087745f4bbae0720f754917000d387bd1ad14c3c9ba3e5721585c49a)

*注意：每个模块都是比较占用内存，记得重启内核*

## 环境准备

运行完后需要`重启内核`，以加载新安装依赖。

In [ ]:
import os

evn='''paddlenlp==2.7.2 
einops
insightface 
onnxruntime 
gradio==3.40.0
munch 
scikit-image 
natsort
regex
'''
with open("./requirements.txt", "w", encoding="utf-8") as fp:
    fp.writelines(evn)

os.system("pip install -q -r requirements.txt --user")


## Demo
用于测试api。

In [ ]:
from PIL import Image

# 生成结果批量保存
def multi_type_images_save(image, fn, suffix='png'):  
    try:
        image = image if not isinstance(image, (np.ndarray, paddle.Tensor)) \
            else Image.fromarray(image.numpy()) if isinstance(image, paddle.Tensor) \
            else Image.fromarray(image) 
        img = load_image(image)
        # 保存图像  
        img.save(fn + '.' + suffix)
    except Exception as e:
        print(e)
    
def batch_save(images, base_fn, suffix='png'): 
    """
    批量保存pil、numpy、pd或path的图片。
    Args：
        images：
            图片，支持pil、numpy、path的单个或者嵌套list对象；
        base_fn：
            保存文件的名称。
    Exmaple：
        from PIL import Image  
        import numpy as np  
        import paddle

        arr = np.random.randint(0, 256, (100, 100, 3), dtype=np.uint8)

        pd = paddle.to_tensor(np.random.randint(0, 256, (100, 100, 3), dtype=np.uint8))
        multi_dtype_images_save(image=arr, fn='test_pd', suffix='png')

        pil = Image.open('test_pd.png')

        images=arr
        batch_save(images, base_fn="test_")
        images=[[arr], pil, pd]
        batch_save(images, base_fn="test_")
    """
    
    if not isinstance(images, list):
        multi_type_images_save(images, base_fn, suffix)
        return 
    for i, image in enumerate(images):  
        base_fn = f"{base_fn.split('_')[0]}_{i}" 
        batch_save(image, base_fn, suffix=suffix)

In [ ]:
# ppdiffusers>=0.24.0"
import numpy as np
import cv2
import re
from PIL import Image
import paddle
from ppdiffusers import (
    ControlNetModel, 
    StableDiffusionXLControlNetPipeline,
    AutoencoderKL, 
	# UNet2DConditionModel, 
	EulerAncestralDiscreteScheduler, 
    KDPM2AncestralDiscreteScheduler, 
    KDPM2DiscreteScheduler
)
from ppdiffusers.utils import load_image, make_image_grid

# load controlnet
controlnet = ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0")

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "SG161222/RealVisXL_V3.0", 
    paddle_dtype=paddle.float16, 
    safety_checker=None, 
    variant="fp16",
    controlnet=controlnet, 
    low_cpu_mem_usage=True
)

# denoise策略
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# 开启 xformers
pipe.enable_xformers_memory_efficient_attention()

# 定义prompt
prompt = [
    "full body shot of a pretty Korean woman, red sweater",
    "<lora:2>Beautiful girl, delicate, peerless face, delicate eyes, off-the-shoulder dress, pink, cherry blossoms, detailed light effect rendering, high-definition picture, beautiful two-dimensional.", 
][-1]
negative_prompt = [
    'low quality, bad quality, sketches, blurry',
    "nsfw",
][-1]

pipe.unfuse_lora() # 每次加载时清空lora
lora = "./static/lora/LYS_真实感模型_SDXL_v1.0.safetensors"  # 模型所在目录
try:  # lora目前无法调整的权重
    # 提取 lora 缩放比例  
    pattern_scale = re.compile(r"<lora:(-?\d*\.?\d*)>")  
    lora_scales = re.findall(pattern_scale, prompt)  
    if lora_scales:  
        lora_scale = float(lora_scales[0])  
    else:  
        lora_scale = 0  
  
    # 移除所有 <*> 格式的内容  
    prompt = re.sub(r"<.*?>", "", prompt)  
  
    # 加载lora权重和设置lora比例
    pipe.load_lora_weights(lora, from_diffusers=True, from_hf_hub=True,)  
    pipe.fuse_lora(lora_scale)  
except Exception as e:  
    print("加载lora时发生错误：", e)
print("prompt:",prompt, "\n", "lora scale is ", lora_scale)


# 随机种子
generator = paddle.Generator().manual_seed(42)

# controlnet权重系数
controlnet_conditioning_scale=0.5

# controlnet参考图
image = load_image(
    "./static/bc.jpg"
)
# controlnet图生成线稿
image = np.array(image)
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
image = Image.fromarray(image)

image = pipe(
    prompt,  
    negative_prompt=negative_prompt, 
    width=1024, 
    height=768, 
    image=image,
    num_inference_steps=30,
    guidance_scale=4,
    num_images_per_prompt=1,
    generator=generator, 
    controlnet_conditioning_scale=controlnet_conditioning_scale, 
).images

# 保存图片
batch_save(image, base_fn='bcsz', suffix='png')

# 生成网格化展示内容
img = make_image_grid(image, rows=1, cols=1)
# 网格化展示
img

In [ ]:
import gc  
del pipe, img, controlnet, image
gc.collect()
paddle.device.cuda.empty_cache()

## 超分

切块处理大图，可以放大任意尺寸。

**StableDiffusion sr**块的大小 `size=(256,256)` ，适合256左右小而模糊的图片`method="sd"`。

**paddle srgan** 块的大小 `size=(1560,1560)`，效率远高于sdsr，适合生成图的放大，默认为`method="df2k"`,

`method="drn"`质量低，但色彩明亮。

In [ ]:
# # WebUI中的超分函数测试["df2k", "drn", "esrgan", "lesr", "sd"]
from PIL import Image
from utils.upscale import upscale_x4

img_path = upscale_x4("./static/bcsz_0.png", method="df2k", size=(512, 512), suffix='jpg', output_dir="./out_puts")
Image.open(img_path[0])

## 模型转换
由torch权重转换而来，这里转换模型后可以在WebUI中使用。

In [ ]:
# # 待转换pytorch模型路径在此修改--多组件权重
# !python convert_pt_2_pd/convert_pt_2_pd.py \
#     --model_name=animagine-xl-2.0


In [ ]:
# # 单文件权重
# !python convert_single_weight_pt_2_pd/convert_key.py \
#     --model_name=/home/aistudio/data/data270713/pony_pencil-XL-v1.0.2.safetensors

In [ ]:
# 模型测测试
import warnings  
warnings.filterwarnings("ignore")

import warnings  
warnings.filterwarnings("ignore")
import paddle, os, sys
from ppdiffusers import (
    StableDiffusionXLPipeline,
    StableDiffusionXLImg2ImgPipeline, 
    EulerAncestralDiscreteScheduler, 
    StableDiffusionXLInpaintPipeline,
    UNet2DConditionModel
)

base_model_path = [
    "./Pony_Pencil-Xl-V1.0.2", 
][-1]

text2img = StableDiffusionXLPipeline.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    paddle_dtype=paddle.float16,
    low_cpu_mem_usage=True, 
)

text2img.scheduler = EulerAncestralDiscreteScheduler.from_config(text2img.scheduler.config)
  
# 使用与text2img相同的组件初始化图像到图像pipeline
img2img = StableDiffusionXLImg2ImgPipeline(**text2img.components)  

# 使用与text2img相同的组件初始化图像修复pipeline
inpaint = StableDiffusionXLInpaintPipeline(**text2img.components)

prompt = "A pretty young woman with silky golden hair that shines with a captivating glow in the sunlight. Her eyes are deep like the sea, revealing boundless wisdom and tenderness. Her face is exquisite, with skin as white as jade, and a confident smile always plays at the corners of her mouth. She wears a simple yet elegant long dress, its hem fluttering gently, as if carrying a touch of ethereal charm. Her posture is upright, her gait composed, and wherever she goes, she becomes the focal point of attention. There's a unique charm about her that involuntarily draws people in."
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"

text2img(
    prompt, 
    negative_prompt=negative_prompt, 
    num_inference_steps=20,  
    guidance_scale=5, 
    # generator= paddle.Generator().manual_seed(42) 
)[0][0]

In [ ]:
import gc
del text2img, img2img, inpaint
gc.collect()
paddle.device.cuda.empty_cache()

<h2><font style="background: linear-gradient(to right,gold, red,pink, green);-webkit-background-clip: text;-webkit-text-fill-color: transparent;">IPAdapterPlusXL</font></h2>

与Photomaker和InstantX/InstantID 功能相似。
模型效果(左图输出，右图输入)

![](https://ai-studio-static-online.cdn.bcebos.com/59f82054242f4d04b433c515ab58e527a2d2901eca94434a97565cf8a5f395ac)


In [ ]:
import warnings  
# 忽略特定警告，调试勿开启此项
warnings.filterwarnings("ignore") 

from PIL import Image
import paddle
import os
from ppdiffusers import (
        StableDiffusionXLPipeline, 
        DDIMScheduler, 
        EulerAncestralDiscreteScheduler,
        StableDiffusionXLImg2ImgPipeline,
    )
from ppdiffusers.utils import load_image, make_image_grid, smart_load
from safetensors.paddle import load_file
# 导入自定义类
from utils.ip_adapter import IPAdapterPlusXL, StableDiffusionXLCustomPipeline

HOME_DIR = os.path.expanduser('~')

# 图片展示
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# 加载模型
base_model_path = ["SG161222/RealVisXL_V3.0", "./Pony_Pencil-Xl-V1.0.2"][-1]
image_encoder_path = [
    "laion/CLIP-ViT-H-14-laion2B-s32B-b79K", 
][-1]
ip_ckpt = [
    os.path.join(HOME_DIR, "data/data264879/ip-adapter-plus-face_sdxl_vit-h.bin"), #挂载的数据集路径
    "ip-adapter/ip-adapter-plus-face_sdxl_vit-h.safetensors",
][-1] 

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
pipe = StableDiffusionXLCustomPipeline.from_pretrained( #plus
    base_model_path, 
    paddle_dtype=paddle.float16, 
    variant="fp16", 
    # scheduler=noise_scheduler,
    low_cpu_mem_usage=True,
)
pipe.enable_xformers_memory_efficient_attention()

ip_model = IPAdapterPlusXL(
    pipe, 
    ip_ckpt=ip_ckpt, 
    image_encoder_path=image_encoder_path,
    device=paddle.get_device(), 
    num_tokens=16
)

In [ ]:
ref_image = Image.open("./static/img_ipadapter.png")
# 只要脸部特征并调整大小(224, 224)
ref_image1 = ref_image.crop((
    ref_image.size[0]*0.1,  # x left
    0,  # y top
    ref_image.size[0]*0.1+ref_image.size[1]*0.45,  # x right
    ref_image.size[1]*0.45  # y bottom
)).resize((224, 224))

imgs = [ref_image, ref_image1]
rows = 1
image_grid(imgs, rows=rows, cols=len(imgs)//rows + (len(imgs)%rows!=0))

paddle.device.cuda.empty_cache()
prompt = [
    "A beautiful girl wearing Sun Dress are loking at viewer in garden",
][-1]
negative_prompt = "low quality, ugly, blurry, opened mouth"

image = ip_model.generate(
    pil_image=ref_image1,  # crop and resize image
    control_guidance_start=0., # 控制姿势，值越小越接近原图
    control_guidance_end=0.8,
    scale=1,  # 值越小与原图差异越大
    prompt=prompt, 
    negative_prompt=negative_prompt,
    num_samples=1, 
    num_inference_steps=30, 
    guidance_scale=5, 
    seed=42,
    eta=0.3, # 只有使用DDIMScheduler才有效
).images
paddle.device.cuda.empty_cache()

# 展示图片
imgs = [image[0], ref_image]
rows = 1
image_grid(imgs, rows=rows, cols=len(imgs)//rows + (len(imgs)%rows!=0))


In [ ]:
import gc
del ip_model, pipe
gc.collect()
paddle.device.cuda.empty_cache()

### Face Adpater


![src+face=result](https://ai-studio-static-online.cdn.bcebos.com/cccacfc3be334672b046f6107516cbf497ad66468594490aa53bc8937d3e873e)


In [ ]:
import warnings  
# 忽略特定警告  
warnings.filterwarnings("ignore") 

import paddle
import os
from ppdiffusers import (
        StableDiffusionXLPipeline, 
        DDIMScheduler, 
        EulerAncestralDiscreteScheduler, 
        StableDiffusionXLImg2ImgPipeline,
        StableDiffusionXLInpaintPipeline
    )
from ppdiffusers.utils import load_image, make_image_grid, smart_load
from safetensors.paddle import load_file
# 导入自定义类
from utils.ip_adapter import IPAdapterPlusXL

HOME_DIR = os.path.expanduser('~')

# 加载模型
base_model_path = ["SG161222/RealVisXL_V3.0", "./Pony_Pencil-Xl-V1.0.2"][-1]
image_encoder_path = [
    "laion/CLIP-ViT-H-14-laion2B-s32B-b79K", 
][-1]
ip_ckpt = [
    os.path.join(HOME_DIR, "data/data264879/ip-adapter-plus-face_sdxl_vit-h.bin"), #挂载的数据集路径
    "ip-adapter/ip-adapter-plus-face_sdxl_vit-h.safetensors",
][-1] 

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained( #plus
    base_model_path, 
    paddle_dtype=paddle.float16, 
    variant="fp16", 
    # scheduler=noise_scheduler,
    low_cpu_mem_usage=True,
)

# pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

pipe.enable_xformers_memory_efficient_attention()

ip_model = IPAdapterPlusXL(
    pipe, 
    ip_ckpt=ip_ckpt, 
    image_encoder_path=image_encoder_path,
    device=paddle.get_device(), 
    num_tokens=16
).generate

In [ ]:
src_image = load_image("./static/text_691436129_ddpm_30_4_0_1712919483.194813.png")
face_image = load_image("./static/img_ipadapter.png")
# 裁剪面部特征
face_image1 = face_image.crop((face_image.size[0]*0.1, 0, face_image.size[0]*0.1+face_image.size[1]*0.42, face_image.size[1]*0.42)).resize((224, 224))

prompt = [
    "A beautiful Chinese girl wearing Sun Dress are loking at viewer",
][-1]
negative_prompt = "low quality, ugly, blurry"

image = ip_model(
    pil_image=face_image1, 
    image=src_image,
    prompt=prompt, 
    negative_prompt=negative_prompt,
    scale=0.75, # 脸部参考图比重
    strength= 0.3, # 重绘步数 = num_inference_steps * strength
    num_samples=1, 
    num_inference_steps=30, 
    guidance_scale=4, 
    seed=42,
).images

paddle.device.cuda.empty_cache()

# 展示图片
from PIL import Image
def image_grid(imgs, rows, cols, figure_size=368):
    assert len(imgs) == rows*cols

    grid = Image.new('RGB', size=(cols*figure_size, rows*figure_size))
    
    for i, img in enumerate(imgs):
        w, h = img.size
        scale = figure_size/max(w, h)
        img = img.resize((int(w*scale), int(h*scale)))
        grid.paste(img, box=(i%cols*figure_size, i//cols*figure_size))
    return grid
imgs = [src_image, face_image, image[0]]
image_grid(imgs, rows=1, cols=len(imgs))

In [ ]:
# 释放内存，GPU有效
import gc
del ip_model, pipe
gc.collect()
paddle.device.cuda.empty_cache()

<h2><font style="background: linear-gradient(to right,gold, red,pink, green);-webkit-background-clip: text;-webkit-text-fill-color: transparent;">WebUI</font></h2>

*先运行测试！*

运行webui前***重启内核***！
双击运行`webui.gradio.py`（controlnet cany使用） 或 `webui_adapter.gradio.py` （Gradio UI的Face Image输入模块点击编辑裁剪出脸部特征）即可,二者只可选一运行。

In [ ]:
# 测试webui
!python webui_adapter.gradio.py

In [ ]:
# 重启内核
!python webui.gradio.py

## Lora 训练

重启内核！

In [ ]:
!git clone https://gitee.com/Covirtue/PaddleMIX.git
# 官方的PaddleMIX中，train_dreambooth_lora_sdxl.py：1643存在bug，
# 每次保存checkpoint后会导致显存不足错误这里使用我修改过的代码

In [ ]:
import os
# lr_scheduler option selected: ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
os.makedirs("lora_trained_xl", exist_ok=True)
!rm -r ./beauty/dlrb/.ipynb_checkpoints
!python PaddleMIX/ppdiffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=stabilityai/stable-diffusion-xl-base-1.0 \
  --instance_data_dir=beauty/dlrb \
  --output_dir=lora_trained_xl \
  --mixed_precision="fp16" \
  --instance_prompt="a shot of customDlrb" \
  --resolution=1024 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1.125e-4 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=10 \
  --max_train_steps=500 \
  --validation_epochs=20 \
  --seed="42" \
  --checkpointing_steps=100 \
  --variant="fp16" \
  --rank=4 # 默认为4(22MB)，32(177MB)或者64(354MB)
  # --validation_prompt="a shot of customDlrb" \
  
  

In [ ]:
# load model
import paddle
from ppdiffusers import (
    AutoencoderKL, 
    DDPMScheduler, 
    EulerAncestralDiscreteScheduler, 
    EulerDiscreteScheduler, 
    KDPM2AncestralDiscreteScheduler, 
    StableDiffusionXLPipeline, 
    UNet2DConditionModel, 
)


pipe = StableDiffusionXLPipeline.from_pretrained(
    # "SG161222/RealVisXL_V3.0", 
    "stabilityai/stable-diffusion-xl-base-1.0", 
    paddle_dtype=paddle.float16, 
    variant="fp16",
    safety_checker=None, 
)

# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
# lora测试
pipe.unfuse_lora()
# 加载lora
pipe.load_lora_weights("lora_trained_xl/checkpoint-500/paddle_lora_weights.safetensors")
# pipe.load_lora_weights("lora_trained_xl/paddle_lora_weights.safetensors")

In [ ]:
prompt = [
    "full body shot, a beautiful woman with delicate face and long hair, wearing red dress.", 
][-1]

prompt_2 = [
    "a shot of customDlrb",
    None, 
][-1]

prompt_1 = [
    prompt, 
    prompt + prompt_2, 
]

negative_prompt = [
    "anything anomaly, bad nails, bad hands:1.2, shortten fingers, bad legs, strech fingers, bad fingers:1.2, bad nails:1.3, low quality, blurry, ugly, bad face, bad eyes, bad toes, bad feet", 
    "username, watermark, texts, anomal, extra, bad nails, nsfw, bad hands, shortten fingers, bad legs, strech fingers, bad fingers:1.2, bad nails:1.3, low quality, blurry, ugly face, bad face, bad eyes, bad toes, bad feet", 
][-1]

In [ ]:
# 两种不同方式的prompt
idx = -1
image1 = pipe(
    prompt=prompt_1[idx], 
    prompt_2=prompt_2[idx], 
    negative_prompt=negative_prompt,
    num_inference_steps=30, 
    width=768, 
    height=1280, 
    guidance_scale=5,
    # generator=paddle.Generator().manual_seed(42)
).images[0]


idx = -2
image2 = pipe(
    prompt=prompt_2[idx], 
    prompt_2=prompt_1[idx], 
    negative_prompt=negative_prompt,
    num_inference_steps=30, 
    width=768, 
    height=1280, 
    guidance_scale=7,
    guidance_rescale=1.3,
    # generator=paddle.Generator().manual_seed(42)
).images[0]

paddle.device.cuda.empty_cache()

from ppdiffusers.utils import image_grid
image_grid([image1, image2], 1, 2)


In [ ]:
# 释放内存，GPU有效
import gc
del pipe
paddle.device.cuda.empty_cache()